## Soportes y Resistencias automaticas de ETH

In [1]:
import requests
import pandas as pd
from datetime import datetime

# Genera el timestamp actual en milisegundos para endTime
end_time = int(datetime.now().timestamp() * 1000)

# Parámetros de la solicitud para ETH/USDT
url = "https://api.binance.com/api/v3/klines"
params = {
    "symbol": "ETHUSDT",       # Cambiado a ETH/USDT
    "interval": "1w",          # Intervalo de velas: '1m', '5m', '1h', '1d', '1w', etc.
    "startTime": 1577836800000, # Tiempo de inicio en formato timestamp (ms)
    "endTime": end_time,       # Tiempo de fin en formato timestamp (ms)
    "limit": 300               # Número máximo de registros a devolver
}

# Realizar la solicitud GET a la API
response = requests.get(url, params=params)
data = response.json()

# Seleccionar solo las primeras 5 columnas (timestamp, open, high, low, close)
filtered_data = [[item[0], item[1], item[2], item[3], item[4]] for item in data]

# Crear el DataFrame de pandas con las columnas necesarias
df = pd.DataFrame(filtered_data, columns=['timestamp', 'open', 'high', 'low', 'close'])

# Convertir la columna 'timestamp' a un formato de fecha legible
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# Convertir las columnas numéricas a tipo float
df[['open', 'high', 'low', 'close']] = df[['open', 'high', 'low', 'close']].astype(float)

# Mostrar los últimos registros del DataFrame
print(df.tail(10))


     timestamp     open     high      low    close
244 2024-09-09  2297.30  2464.82  2272.80  2316.10
245 2024-09-16  2316.09  2632.57  2252.39  2581.00
246 2024-09-23  2580.99  2728.60  2539.49  2657.62
247 2024-09-30  2657.62  2663.50  2310.00  2440.03
248 2024-10-07  2440.02  2521.00  2330.66  2468.91
249 2024-10-14  2468.92  2759.00  2443.39  2746.91
250 2024-10-21  2746.91  2769.48  2382.59  2507.80
251 2024-10-28  2507.80  2722.30  2411.00  2457.73
252 2024-11-04  2457.73  3248.52  2357.59  3183.21
253 2024-11-11  3183.20  3442.50  3105.00  3276.72


In [2]:
# Definir una ventana para buscar máximos y mínimos locales
window = 10  # Ajusta el tamaño de la ventana según el nivel de detalle que desees

# Listas para almacenar los niveles de soporte y resistencia
resistencias = []
soportes = []

# Recorrer los datos de precios
for i in range(window, len(df) - window):
    max_local = max(df['high'][i - window:i + window + 1])
    min_local = min(df['low'][i - window:i + window + 1])

    # Si el precio actual es un máximo local, se marca como resistencia
    if df['high'][i] == max_local:
        resistencias.append((df['timestamp'][i], df['high'][i]))
    
    # Si el precio actual es un mínimo local, se marca como soporte
    if df['low'][i] == min_local:
        soportes.append((df['timestamp'][i], df['low'][i]))

# Crear DataFrames para soportes y resistencias
df_resistencias = pd.DataFrame(resistencias, columns=['timestamp', 'resistencia'])
df_soportes = pd.DataFrame(soportes, columns=['timestamp', 'soporte'])

# Mostrar los primeros resultados
print("Resistencias detectadas:")
print(df_resistencias.head())

print("\nSoportes detectados:")
print(df_soportes.head())


Resistencias detectadas:
   timestamp  resistencia
0 2020-08-31       488.84
1 2021-05-10      4372.72
2 2021-11-08      4868.00
3 2022-03-28      3580.34
4 2022-08-08      2030.00

Soportes detectados:
   timestamp  soporte
0 2021-06-21  1700.48
1 2022-01-24  2159.00
2 2022-06-13   881.56
3 2022-11-07  1073.53
4 2023-10-09  1521.00


In [3]:
import plotly.graph_objects as go

# Crear una figura para el gráfico de velas
fig = go.Figure(data=[go.Candlestick(
    x=df['timestamp'],
    open=df['open'],
    high=df['high'],
    low=df['low'],
    close=df['close']
)])

# Agregar líneas horizontales para cada resistencia
for resistencia in df_resistencias['resistencia']:
    fig.add_shape(
        type="line",
        x0=df['timestamp'].min(), y0=resistencia,
        x1=df['timestamp'].max(), y1=resistencia,
        line=dict(color="red", width=1.5, dash="dot"),
        name="Resistencia"
    )

# Agregar líneas horizontales para cada soporte
for soporte in df_soportes['soporte']:
    fig.add_shape(
        type="line",
        x0=df['timestamp'].min(), y0=soporte,
        x1=df['timestamp'].max(), y1=soporte,
        line=dict(color="green", width=1.5, dash="dot"),
        name="Soporte"
    )

# Configurar el diseño del gráfico con un mayor tamaño y título actualizado para ETH
fig.update_layout(
    title="Gráfico de velas ETH/USDT",
    xaxis_title="Fecha",
    yaxis_title="Precio (USD)",
    xaxis_rangeslider_visible=False,  # Oculta el deslizador de rango
    width=1500,  # Ancho del gráfico
    height=800   # Altura del gráfico
)

# Mostrar el gráfico
fig.show()
